In [1]:
# pip install PyPDF2
# pip install PDFFileReader
# pip install pdf

import PyPDF2
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import os

filename = "NH투자증권.pdf"
filepath = os.path.join("../pdf_to_text_min",filename)

# PDF의 Total 페이지를 읽어오기 
fp = open(filepath, 'rb')
total_pages = PyPDF2.PdfFileReader(fp).numPages

# pdfminer로 페이지별 텍스트 가져오기 
page_text = {}
for page_no in range(total_pages):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    device = TextConverter(rsrcmgr, retstr, laparams=LAParams())
    fb = open(filepath, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    caching = True
    maxpages = 0
    pagenos = [page_no]
    for page in PDFPage.get_pages(fb, pagenos, maxpages = maxpages, 
                              caching = caching, check_extractable = True):
        interpreter.process_page(page)
        page_text[page_no] = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    
#print(page_text[1][:-1])
"""
# 원하는 페이지 첫번째 []에 입력 
# 1 page 내용 출력
#print(page_text[1])
#print(page_text[1][:-1])

print("Start")
with open("NH_paper.txt", 'w',encoding="UTF-8") as f:
    for idx in range(len(page_text)):
        f.write(page_text[idx])
print("Finish")
"""

'\n# 원하는 페이지 첫번째 []에 입력 \n# 1 page 내용 출력\n#print(page_text[1])\n#print(page_text[1][:-1])\n\nprint("Start")\nwith open("NH_paper.txt", \'w\',encoding="UTF-8") as f:\n    for idx in range(len(page_text)):\n        f.write(page_text[idx])\nprint("Finish")\n'

In [2]:
import pandas as pd
import tabula
import re

page_num = 0
preproReport = []

while page_num != len(page_text):

    # pdf에 있는 table을 탐지
    df = tabula.read_pdf(filepath, pages = str(page_num + 1), multiple_tables = True, guess = True)

    # page_num 쪽의 내용을 리스트로 변환
    page_content = page_text[page_num]
    pageTolist = re.split('[ /\n]+', page_content)

    print("Working on page(PDF): ",page_num+1)
    # 탐지한 table에 대해서 전처리 작업
    # 열 이름이 Unnamed:~ 로 탐지되는 것들은 쓸모 없는 것들이 많아서 해당 열을 삭제
    for table_num in range(len(df)):
        table = df[table_num].loc[:,df[table_num].columns.str.startswith('Unnamed:') == False]
        # 탐지한 table이 데이터프레인 형태인데, 이를 문자열로 변환하며 개행문자"\n" 제거
        tableToStr = table.to_string().replace("\n","")
        # 변환한 문자열을 공백을 기준으로 나눠 리스트에 담아준다
        tableTolist = tableToStr.split(" ")
        # 리스트로 담아주면서 의미없는 공백과 NaN을 제거
        tableTolist = [i for i in tableTolist if i != '' and i != 'NaN']
        print("테이블 길이: ",len(tableTolist))

        # table이 pdf의 페이지에서 어디에 위치하는지 찾는 작업 시작
        # page_num 쪽의 내용을 리스트로 변환한 pageTolist에서, table의 내용이 몇 번째 인덱스에 위치하는지 찾는다.
        # pageTolist의 table_start번째 인덱스부터 table_finish번째 인덱스까지 한칸씩 이동하며, table의 내용이 pageTolist의 어느 인덱스에 위치하는지 찾는다.
        # 최적의 인덱스 위치를 찾기 위해, 유사도를 확인한다.
        # 여기서 유사도란, tableTolist와 pageTolist[table_start:table_finish]이 유사한 정도를 의미한다 (교집합)
        table_start = 0
        table_finish = len(tableTolist)- 1
        simil = 0
        best_start = 0
        best_finish = 0

        while table_finish < len(pageTolist):
            simil_temp = len(set(pageTolist[table_start:table_finish]).intersection(tableTolist)) / len(tableTolist)
            if simil_temp > simil:
                simil = simil_temp
                best_start = table_start
                best_finish = table_finish
            table_start += 1
            table_finish += 1
        del pageTolist[best_start:best_finish+1]
        print("Table number: ",table_num+1)
    preproReport.append(pageTolist)
    page_num += 1 

Working on page(PDF):  1
Working on page(PDF):  2
Working on page(PDF):  3
Working on page(PDF):  4
Working on page(PDF):  5
테이블 길이:  12
Table number:  1
테이블 길이:  10
Table number:  2
Working on page(PDF):  6
테이블 길이:  67
Table number:  1
Working on page(PDF):  7
Working on page(PDF):  8
테이블 길이:  179
Table number:  1
Working on page(PDF):  9
테이블 길이:  536
Table number:  1
Working on page(PDF):  10
Working on page(PDF):  11
Working on page(PDF):  12
테이블 길이:  53
Table number:  1
Working on page(PDF):  13
Working on page(PDF):  14
Working on page(PDF):  15
Working on page(PDF):  16
Working on page(PDF):  17
Working on page(PDF):  18
Working on page(PDF):  19
테이블 길이:  169
Table number:  1
Working on page(PDF):  20
테이블 길이:  146
Table number:  1
Working on page(PDF):  21
Working on page(PDF):  22
테이블 길이:  343
Table number:  1
Working on page(PDF):  23
Working on page(PDF):  24
테이블 길이:  323
Table number:  1
Working on page(PDF):  25
테이블 길이:  247
Table number:  1
Working on page(PDF):  26
Working

In [17]:
# NH 투자증권 보고서 19페이지
preproReport[22]

['023',
 'INTRODUCTION',
 'KEY',
 'HIGHLIGHTS',
 'TOWARDS',
 'THE',
 'ESG',
 'LEADERSHIP',
 'TOWARDS',
 'A',
 'SUSTAINABLE',
 'FUTURE',
 'APPENDIX',
 'NH임직원',
 '고충처리',
 '제도',
 'NH투자증권은',
 '임직원에',
 '대한',
 '상시적인',
 '고충처리를',
 '위해',
 '사내',
 '홈페이지의',
 '‘소통과',
 '공감’',
 '코너를',
 'CEO',
 '직통',
 '소통채널로',
 '운영하고',
 '있습니다.',
 '2021년',
 '한',
 '해',
 '동안',
 '총',
 '36개의',
 '자유의견과',
 '고충사항이',
 '게시되면서',
 '크게',
 '활성화되었으며,',
 '임직원들은',
 '이',
 '공간에서',
 '서로의',
 '고충을',
 '공유하고',
 '질의,',
 '답변하고',
 '있습니다.',
 '아울러',
 '각',
 '주제에',
 '부합하는',
 '담당부서는',
 '직접',
 '답변을',
 '게시하여',
 '임직원들의',
 '고충을',
 '귀담아듣고',
 '실효적으로',
 '해소하기',
 '위해',
 '노력하고',
 '있습니다.',
 '이외에도',
 '임직원들은',
 '사내',
 '인사관리',
 '시스템',
 '‘HRIS’를',
 '통해',
 '고충에',
 '대해',
 '자유롭게',
 '제보하고',
 '인사본부',
 '내',
 '고충처리',
 '담당직원으로부터',
 '개인적인',
 '상담을',
 '제공받을',
 '수',
 '있습니다.',
 '임직원',
 '인권교육',
 '프로그램',
 'NH투자증권은',
 '관계법령에',
 '의거하여',
 '임직원의',
 '직장',
 '내',
 '인권보호',
 '및',
 '인식개선을',
 '위한',
 '인권교육을',
 '정기적으로',
 '실시합니다.',
 '인권교육은',
 '전',
 '임직원을',
 '대상으로',
 '하며,',
 '성희롱',
 '예방교육',
 